In [3]:
import PIL
from PIL import Image
import numpy as np
import os 
import random
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.optimizers import SGD, Adam
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

from keras.wrappers.scikit_learn import KerasClassifier
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K
from keras import regularizers

In [4]:
dirs = ['/media/jan/Volume_big/datasets/Second_try/Grid_size_874_binary/3layer/no_porosity',
        '/media/jan/Volume_big/datasets/Second_try/Grid_size_874_binary/3layer/porosity']

length = 2000
image_paths = []
labels = []
data = []

In [5]:
height = 128
width = 128
depth = 3
inputShape = (height, width, depth)

In [7]:
for d in dirs: 
    cur_paths = []
    for path in os.listdir(d):
        full_path = os.path.join(d,path)
        cur_paths.append(full_path)
        
    random.seed(42)
    random.shuffle(cur_paths)
    cur_paths = cur_paths[:length]
    
    image_paths.append(cur_paths)        

image_paths = [val for sublist in image_paths for val in sublist]
random.seed(42)
random.shuffle(image_paths)

######
for path in image_paths:
    label = path.split(os.path.sep)[-2]
    labels.append(label)
    
    array = np.load(path)
    img = Image.fromarray(array)
    img = img.resize((height, width), PIL.Image.LANCZOS)
    array_resized_flattened = np.array(img)
    data.append(array_resized_flattened)
    
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=42)

lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
trainY = np.hstack((trainY, 1- trainY))

testY = lb.transform(testY)
testY = np.hstack((testY, 1- testY))

In [8]:
def create_model(lr = 0.001):
    
    model = Sequential()
    model.add(Conv2D(3,(6,6), padding="same", input_shape = inputShape))   #, kernel_regularizer=regularizers.l2(0.01)))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis = -1))
    model.add(MaxPooling2D(pool_size = (2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(6, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=-1))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(32))
    model.add(Activation("relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    
    model.add(Dense(2))
    model.add(Activation("sigmoid"))
    
    optimizer = Adam(lr = lr)
    model.compile(loss = 'binary_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
    return model 

In [9]:
trainY.shape

(3000, 2)

In [10]:
trainY[1]

array([1, 0])

In [11]:
model = KerasClassifier(build_fn=create_model)

In [12]:
batch_size = [8, 16, 32]
epochs = [20, 40, 60 ]
learning_rate = [0.001, 0.01, 0.1]
param_grid = dict(batch_size=batch_size, epochs=epochs, lr = learning_rate)

In [71]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=3)

In [72]:
grid_result = grid.fit(trainX, trainY)

Epoch 1/20
2000/2000 [==============================] - 25s 13ms/step - loss: 0.6886 - acc: 0.6245
Epoch 2/20
2000/2000 [==============================] - 6s 3ms/step - loss: 0.5992 - acc: 0.6813
Epoch 3/20
2000/2000 [==============================] - 6s 3ms/step - loss: 0.5891 - acc: 0.6892
Epoch 4/20
2000/2000 [==============================] - 6s 3ms/step - loss: 0.5751 - acc: 0.6973
Epoch 5/20
2000/2000 [==============================] - 6s 3ms/step - loss: 0.5669 - acc: 0.7010
Epoch 6/20
2000/2000 [==============================] - 6s 3ms/step - loss: 0.5564 - acc: 0.7045
Epoch 7/20
2000/2000 [==============================] - 6s 3ms/step - loss: 0.5497 - acc: 0.7160
Epoch 8/20
2000/2000 [==============================] - 6s 3ms/step - loss: 0.5395 - acc: 0.7173
Epoch 9/20
2000/2000 [==============================] - 6s 3ms/step - loss: 0.5609 - acc: 0.7115
Epoch 10/20
2000/2000 [==============================] - 6s 3ms/step - loss: 0.5428 - acc: 0.7135
Epoch 11/20
2000/2000 [====

2000/2000 [==============================] - 6s 3ms/step - loss: 0.6281 - acc: 0.6817
Epoch 4/20
2000/2000 [==============================] - 6s 3ms/step - loss: 0.6534 - acc: 0.6348
Epoch 5/20
2000/2000 [==============================] - 6s 3ms/step - loss: 0.6425 - acc: 0.6663
Epoch 6/20
2000/2000 [==============================] - 6s 3ms/step - loss: 0.6461 - acc: 0.6623
Epoch 7/20
2000/2000 [==============================] - 6s 3ms/step - loss: 0.6376 - acc: 0.6613
Epoch 8/20
2000/2000 [==============================] - 6s 3ms/step - loss: 0.6317 - acc: 0.6605
Epoch 9/20
2000/2000 [==============================] - 6s 3ms/step - loss: 0.6403 - acc: 0.6590
Epoch 10/20
2000/2000 [==============================] - 6s 3ms/step - loss: 0.5964 - acc: 0.6887
Epoch 11/20
2000/2000 [==============================] - 6s 3ms/step - loss: 0.6183 - acc: 0.6790
Epoch 12/20
2000/2000 [==============================] - 6s 3ms/step - loss: 0.6570 - acc: 0.6440
Epoch 13/20
2000/2000 [===============

2000/2000 [==============================] - 6s 3ms/step - loss: 0.5750 - acc: 0.6943
Epoch 28/40
2000/2000 [==============================] - 6s 3ms/step - loss: 0.5625 - acc: 0.7035
Epoch 29/40
2000/2000 [==============================] - 6s 3ms/step - loss: 0.5569 - acc: 0.7160
Epoch 30/40
2000/2000 [==============================] - 6s 3ms/step - loss: 0.5615 - acc: 0.7035
Epoch 31/40
2000/2000 [==============================] - 6s 3ms/step - loss: 0.5621 - acc: 0.7125
Epoch 32/40
2000/2000 [==============================] - 6s 3ms/step - loss: 0.5618 - acc: 0.7050
Epoch 33/40
2000/2000 [==============================] - 6s 3ms/step - loss: 0.5627 - acc: 0.7135
Epoch 34/40
2000/2000 [==============================] - 6s 3ms/step - loss: 0.5462 - acc: 0.7165
Epoch 35/40
2000/2000 [==============================] - 6s 3ms/step - loss: 0.5626 - acc: 0.7185
Epoch 36/40
2000/2000 [==============================] - 6s 3ms/step - loss: 0.5599 - acc: 0.7130
Epoch 37/40
2000/2000 [=========

2000/2000 [==============================] - 6s 3ms/step - loss: 0.6212 - acc: 0.6665
Epoch 32/40
2000/2000 [==============================] - 6s 3ms/step - loss: 0.6416 - acc: 0.6730
Epoch 33/40
2000/2000 [==============================] - 6s 3ms/step - loss: 0.5992 - acc: 0.6825
Epoch 34/40
2000/2000 [==============================] - 6s 3ms/step - loss: 0.6247 - acc: 0.6720
Epoch 35/40
2000/2000 [==============================] - 6s 3ms/step - loss: 0.6171 - acc: 0.6770
Epoch 36/40
2000/2000 [==============================] - 6s 3ms/step - loss: 0.6317 - acc: 0.6560
Epoch 37/40
2000/2000 [==============================] - 6s 3ms/step - loss: 0.6133 - acc: 0.6950
Epoch 38/40
2000/2000 [==============================] - 6s 3ms/step - loss: 0.6216 - acc: 0.6825
Epoch 39/40
2000/2000 [==============================] - 6s 3ms/step - loss: 0.6022 - acc: 0.6880
Epoch 40/40
1000/1000 [==============================] - 18s 18ms/step
Epoch 1/40
2000/2000 [==============================] - 44s

2000/2000 [==============================] - 6s 3ms/step - loss: 0.2114 - acc: 0.9165
Epoch 57/60
2000/2000 [==============================] - 6s 3ms/step - loss: 0.2064 - acc: 0.9193
Epoch 58/60
2000/2000 [==============================] - 6s 3ms/step - loss: 0.1959 - acc: 0.9293
Epoch 59/60
2000/2000 [==============================] - 6s 3ms/step - loss: 0.2024 - acc: 0.9260
Epoch 60/60
1000/1000 [==============================] - 20s 20ms/step
Epoch 1/60
2000/2000 [==============================] - 47s 24ms/step - loss: 0.6598 - acc: 0.6655
Epoch 2/60
2000/2000 [==============================] - 6s 3ms/step - loss: 0.6157 - acc: 0.6775
Epoch 3/60
2000/2000 [==============================] - 6s 3ms/step - loss: 0.5898 - acc: 0.6837
Epoch 4/60
2000/2000 [==============================] - 6s 3ms/step - loss: 0.5764 - acc: 0.7020
Epoch 5/60
2000/2000 [==============================] - 6s 3ms/step - loss: 0.5508 - acc: 0.7142
Epoch 6/60
2000/2000 [==============================] - 6s 3ms

...

2000/2000 [==============================] - 2s 1ms/step - loss: 0.5503 - acc: 0.7020
Epoch 4/40
2000/2000 [==============================] - 2s 1ms/step - loss: 0.5324 - acc: 0.7190
Epoch 5/40
2000/2000 [==============================] - 2s 1ms/step - loss: 0.5315 - acc: 0.7182
Epoch 6/40
2000/2000 [==============================] - 2s 1ms/step - loss: 0.5334 - acc: 0.7133
Epoch 7/40
2000/2000 [==============================] - 2s 1ms/step - loss: 0.5237 - acc: 0.7170
Epoch 8/40
2000/2000 [==============================] - 2s 1ms/step - loss: 0.5237 - acc: 0.7253
Epoch 9/40
2000/2000 [==============================] - 2s 1ms/step - loss: 0.5217 - acc: 0.7198
Epoch 10/40
2000/2000 [==============================] - 2s 1ms/step - loss: 0.5173 - acc: 0.7125
Epoch 11/40
2000/2000 [==============================] - 2s 1ms/step - loss: 0.5175 - acc: 0.7165
Epoch 12/40
2000/2000 [==============================] - 2s 1ms/step - loss: 0.5197 - acc: 0.7328
Epoch 13/40
2000/2000 [===============

2000/2000 [==============================] - 2s 1ms/step - loss: 0.5593 - acc: 0.6885
Epoch 8/40
2000/2000 [==============================] - 2s 1ms/step - loss: 0.5536 - acc: 0.7023
Epoch 9/40
2000/2000 [==============================] - 2s 1ms/step - loss: 0.5646 - acc: 0.6933
Epoch 10/40
2000/2000 [==============================] - 2s 1ms/step - loss: 0.5517 - acc: 0.6870
Epoch 11/40
2000/2000 [==============================] - 2s 1ms/step - loss: 0.5669 - acc: 0.6860
Epoch 12/40
2000/2000 [==============================] - 2s 1ms/step - loss: 0.5671 - acc: 0.6757
Epoch 13/40
2000/2000 [==============================] - 2s 1ms/step - loss: 0.5876 - acc: 0.6893
Epoch 14/40
2000/2000 [==============================] - 2s 1ms/step - loss: 0.5706 - acc: 0.7027
Epoch 15/40
2000/2000 [==============================] - 2s 1ms/step - loss: 0.5416 - acc: 0.6970
Epoch 16/40
2000/2000 [==============================] - 2s 1ms/step - loss: 0.5444 - acc: 0.6975
Epoch 17/40
2000/2000 [===========

2000/2000 [==============================] - 2s 1ms/step - loss: 0.0681 - acc: 0.9800
Epoch 33/60
2000/2000 [==============================] - 2s 1ms/step - loss: 0.0693 - acc: 0.9725
Epoch 34/60
2000/2000 [==============================] - 2s 1ms/step - loss: 0.0638 - acc: 0.9770
Epoch 35/60
2000/2000 [==============================] - 2s 1ms/step - loss: 0.0600 - acc: 0.9838
Epoch 36/60
2000/2000 [==============================] - 2s 1ms/step - loss: 0.0572 - acc: 0.9790
Epoch 37/60
2000/2000 [==============================] - 2s 1ms/step - loss: 0.0539 - acc: 0.9818
Epoch 38/60
2000/2000 [==============================] - 2s 1ms/step - loss: 0.0654 - acc: 0.9765
Epoch 39/60
2000/2000 [==============================] - 2s 1ms/step - loss: 0.0528 - acc: 0.9862
Epoch 40/60
2000/2000 [==============================] - 2s 1ms/step - loss: 0.0556 - acc: 0.9805
Epoch 41/60
2000/2000 [==============================] - 2s 1ms/step - loss: 0.0666 - acc: 0.9765
Epoch 42/60
2000/2000 [=========

2000/2000 [==============================] - 2s 1ms/step - loss: 0.5053 - acc: 0.7202
Epoch 18/60
2000/2000 [==============================] - 2s 1ms/step - loss: 0.4945 - acc: 0.7262
Epoch 19/60
2000/2000 [==============================] - 2s 1ms/step - loss: 0.5050 - acc: 0.7185
Epoch 20/60
2000/2000 [==============================] - 2s 1ms/step - loss: 0.4949 - acc: 0.7157
Epoch 21/60
2000/2000 [==============================] - 2s 1ms/step - loss: 0.4946 - acc: 0.7248
Epoch 22/60
2000/2000 [==============================] - 2s 1ms/step - loss: 0.4869 - acc: 0.7412
Epoch 23/60
2000/2000 [==============================] - 2s 1ms/step - loss: 0.4899 - acc: 0.7283
Epoch 24/60
2000/2000 [==============================] - 2s 1ms/step - loss: 0.4839 - acc: 0.7385
Epoch 25/60
2000/2000 [==============================] - 2s 1ms/step - loss: 0.4709 - acc: 0.7433
Epoch 26/60
2000/2000 [==============================] - 2s 1ms/step - loss: 0.4880 - acc: 0.7278
Epoch 27/60
2000/2000 [=========

2000/2000 [==============================] - 2s 1ms/step - loss: 0.5670 - acc: 0.6705
Epoch 3/60
2000/2000 [==============================] - 2s 1ms/step - loss: 0.5677 - acc: 0.6745
Epoch 4/60
2000/2000 [==============================] - 2s 1ms/step - loss: 0.5686 - acc: 0.6753
Epoch 5/60
2000/2000 [==============================] - 2s 1ms/step - loss: 0.5610 - acc: 0.6947
Epoch 6/60
2000/2000 [==============================] - 2s 1ms/step - loss: 0.5593 - acc: 0.6910
Epoch 7/60
2000/2000 [==============================] - 2s 1ms/step - loss: 0.5619 - acc: 0.6827
Epoch 8/60
2000/2000 [==============================] - 2s 1ms/step - loss: 0.5612 - acc: 0.6823
Epoch 9/60
2000/2000 [==============================] - 2s 1ms/step - loss: 0.5638 - acc: 0.6967
Epoch 10/60
2000/2000 [==============================] - 2s 1ms/step - loss: 0.5548 - acc: 0.6793
Epoch 11/60
2000/2000 [==============================] - 2s 1ms/step - loss: 0.5588 - acc: 0.6860
Epoch 12/60
2000/2000 [================

In [73]:
grid_result.cv_results_

{'mean_fit_time': array([141.47007179, 145.63643487, 151.35148899, 277.89289331,
        284.34662247, 289.94936728, 424.78170013, 436.66110778,
        443.58709844, 124.67103036, 127.54852049, 130.30682405,
        208.80620495, 212.14615107, 215.4132882 , 295.43077612,
        301.24193923, 304.72167555, 131.3177518 , 124.14474297,
        122.46016343, 171.52321712, 174.33398549, 177.82403827,
        230.66612347, 235.99321858, 241.36415434]),
 'std_fit_time': array([1.87815699, 1.47745913, 1.8056851 , 2.51071966, 2.55825982,
        2.24269356, 1.6655141 , 4.68088544, 2.00099689, 1.02194401,
        1.64873239, 1.15976541, 3.02514666, 0.7621146 , 1.12226714,
        0.73459323, 1.48607702, 0.9496137 , 7.96939524, 0.69321191,
        1.13865016, 0.5288001 , 1.02755525, 1.60486338, 1.88075523,
        1.55787453, 1.85320008]),
 'mean_score_time': array([11.98515344, 13.42360997, 14.396245  , 15.7562379 , 17.09110053,
        18.64165839, 20.02526283, 22.1250697 , 24.42755111, 24.18

In [74]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.714833 using {'batch_size': 32, 'epochs': 20, 'lr': 0.1}
0.713500 (0.005788) with: {'batch_size': 8, 'epochs': 20, 'lr': 0.001}
0.700333 (0.025332) with: {'batch_size': 8, 'epochs': 20, 'lr': 0.01}
0.699333 (0.015195) with: {'batch_size': 8, 'epochs': 20, 'lr': 0.1}
0.693333 (0.007717) with: {'batch_size': 8, 'epochs': 40, 'lr': 0.001}
0.703333 (0.009031) with: {'batch_size': 8, 'epochs': 40, 'lr': 0.01}
0.694000 (0.006976) with: {'batch_size': 8, 'epochs': 40, 'lr': 0.1}
0.707167 (0.013306) with: {'batch_size': 8, 'epochs': 60, 'lr': 0.001}
0.709667 (0.006549) with: {'batch_size': 8, 'epochs': 60, 'lr': 0.01}
0.680333 (0.038904) with: {'batch_size': 8, 'epochs': 60, 'lr': 0.1}
0.672000 (0.010464) with: {'batch_size': 16, 'epochs': 20, 'lr': 0.001}
0.714000 (0.011683) with: {'batch_size': 16, 'epochs': 20, 'lr': 0.01}
0.695333 (0.022691) with: {'batch_size': 16, 'epochs': 20, 'lr': 0.1}
0.682000 (0.011188) with: {'batch_size': 16, 'epochs': 40, 'lr': 0.001}
0.682000 (0.012570) 